# Lending Club EDA

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Reading the data into a Pandas df
df = pd.read_csv("data/loan.csv")

C:\Users\abhir\AppData\Local\Temp\ipykernel_8800\3598531694.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/loan.csv")


In [3]:
# Inspecting the data
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
df.shape

(39717, 111)

In [5]:
# Dropping unnecessary id columns
df.drop(['id', 'member_id'], axis = 1, inplace = True)
df.shape

(39717, 109)

#### For the purpose of analysis, we will not consider the loans which are still in Current status since we cannot consider them to be either default or not default

In [6]:
df = df[df['loan_status'] != 'Current']
df.shape

(38577, 109)

In [7]:
# A function to retrieve the data types for each column in a Pandas DataFrame
def get_data_types(df):
    data_types = {}
    for (col, dtype) in zip(df.columns, df.dtypes):
        data_types[col] = dtype
    
    columns = []
    dtypes = []
    for key,value in data_types.items():
        columns.append(key)
        dtypes.append(value)
    
    data = {"column_name":columns,"data_type":dtypes}
    data_types = pd.DataFrame(data)
    return data_types

In [8]:
# A function to retrieve the null counts for each column in a Pandas DataFrame
def get_null_counts(df):
    nulls = {}
    for (col, dtype) in zip(df.columns, df.isnull().sum()):
        nulls[col] = dtype

    columns = []
    count_nulls = []
    for key,value in nulls.items():
        columns.append(key)
        count_nulls.append(value)
    data = {"column_name":columns,"count_nulls":count_nulls}
    nulls = pd.DataFrame(data)
    return nulls

In [9]:
# A funciton to print two values passed as arguments along with the percentage difference between them
def print_vals(x,y):
    print(f"df1:{x}")
    print(f"df2:{y}")
    diff = ((y-x)/x)*100
    print(f"difference = {diff}")

In [10]:
# A function to compare descriptive stats for the same column in two Pandas DataFrames
def compare_numerical_column(df1, df2, column_name):
    arr1 = np.array(df1[column_name])
    arr2 = np.array(df2[column_name])
    # All value differences are calculated as (((value in df2)-(value in df1))/(value in df1))*100
    print("Min:")
    print_vals(np.min(arr1),np.min(arr2))
    print("Max:")
    print_vals(np.max(arr1),np.max(arr2))
    print("Mean:")
    print_vals(np.mean(arr1),np.mean(arr2))
    print("25th percentile:")
    print_vals(np.percentile(arr1, 25),np.percentile(arr2, 25))
    print("Median:")
    print_vals(np.median(arr1),np.median(arr2))
    print("75th percentile:")
    print_vals(np.percentile(arr1, 75),np.percentile(arr2, 75))

In [11]:
# A function to plot side by side boxplots and then histograms for the same column in two Pandas DataFrames
def plot_numerical_cols(df1, df2, column_name):
    x = 'loan_status'
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Side-by-side boxplot
    axs[0, 0].boxplot([df1[column_name], df2[column_name]], patch_artist=True, boxprops=dict(facecolor='lightblue'))
    axs[0, 0].set_xticklabels([f'{df1[x][0]}', f'{df2[x][0]}'])
    axs[0, 0].set_ylabel(column_name)
    axs[0, 0].set_title('Side-by-Side Boxplot')

    axs[1, 0].hist(df1[column_name], bins=10, alpha=0.5, color='skyblue', label=f'Histogram ({df1[x][0]})')
    axs[1, 0].set_xlabel(column_name)
    axs[1, 0].set_ylabel('Frequency')
    axs[1, 0].set_title(f"{df1[x][0]}")

    axs[1, 1].hist(df2[column_name], bins=10, alpha=0.5, color='lightgreen', label=f'Histogram ({df2[x][0]})')
    axs[1, 1].set_xlabel(column_name)
    axs[1, 1].set_ylabel('Frequency')
    axs[1, 1].set_title(f"{df2[x][0]}")

    plt.subplots_adjust(hspace=0.3, wspace=0.3)
    plt.show()

In [12]:
# A function to remove outliers from a numerical column
# Outliers are defined as values lying outside -3 standard deviations to +3 standard deviations from the mean
def remove_outliers(df,col):
    arr = np.array(df[col])
    mean = np.mean(arr)
    std = np.std(arr)
    df = df[(df[col] <= mean+(3*std)) & (df[col >= mean-(3*std)])]
    return df

In [13]:
def get_metadata(df):
    data_types = get_data_types(df)
    nulls = get_null_counts(df)
    df_metadata = pd.merge(data_types,nulls, on = 'column_name', how = 'inner')
    return df_metadata

#### Discovering and dealing with null/missing values

In [14]:
df_metadata = get_metadata(df)

In [15]:
# Calculating percentage of missing values
df_metadata['null_percentage'] = (df_metadata['count_nulls']/df.shape[0])*100

In [16]:
df_metadata['null_percentage'].value_counts()

100.000000    55
0.000000      41
0.145164       2
6.185033       1
2.677761       1
32.472717      1
0.028514       1
64.559193      1
92.897322      1
0.129611       1
0.184047       1
0.005184       1
1.806776       1
0.101097       1
Name: null_percentage, dtype: int64

In [18]:
df_metadata

,column_name,data_type,count_nulls,null_percentage
17,desc,object,12527,32.472717
26,mths_since_last_delinq,float64,24905,64.559193
27,mths_since_last_record,float64,35837,92.897322
45,next_pymnt_d,object,38577,100.000000
48,mths_since_last_major_derog,float64,38577,100.000000
51,annual_inc_joint,float64,38577,100.000000
52,dti_joint,float64,38577,100.000000
53,verification_status_joint,float64,38577,100.000000
55,tot_coll_amt,float64,38577,100.000000
56,tot_cur_bal,float64,38577,100.000000


In [17]:
# If a column has 10% or more of it's values missing, replacing them may skew the data and, therefore, the column is dropped
df_metadata = df_metadata[df_metadata['null_percentage'] >= 10]

In [19]:
columns_to_remove = df_metadata['column_name'].to_list()
len(columns_to_remove)

58

In [20]:
df.drop(columns_to_remove, axis = 1, inplace = True)
df.shape

(38577, 51)

In [21]:
df_metadata = get_metadata(df)

In [22]:
df_metadata

,column_name,data_type,count_nulls
0,loan_amnt,int64,0
1,funded_amnt,int64,0
2,funded_amnt_inv,float64,0
3,term,object,0
4,int_rate,object,0
5,installment,float64,0
6,grade,object,0
7,sub_grade,object,0
8,emp_title,object,2386
9,emp_length,object,1033


In [23]:
df_metadata['data_type'].value_counts()

object     22
float64    18
int64      11
Name: data_type, dtype: int64

In [24]:
numeric = df_metadata[df_metadata['data_type'] != 'object']
non_numeric = df_metadata[df_metadata['data_type'] == 'object']

In [25]:
numeric_cols = numeric['column_name'].to_list()
len(numeric_cols)

29

In [26]:
non_numeric_cols = non_numeric['column_name'].to_list()
len(non_numeric_cols)

22

In [30]:
df_obj = df[non_numeric_cols]

In [31]:
df_obj.head(10)

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,...,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type
1,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-11,Charged Off,...,car,bike,309xx,GA,Apr-99,9.40%,f,Apr-13,Sep-13,INDIVIDUAL
3,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-11,Fully Paid,...,other,personel,917xx,CA,Feb-96,21%,f,Jan-15,Apr-16,INDIVIDUAL
5,36 months,7.90%,A,A4,Veolia Transportaton,3 years,RENT,Source Verified,Dec-11,Fully Paid,...,wedding,My wedding loan I promise to pay back,852xx,AZ,Nov-04,28.30%,f,Jan-15,Jan-16,INDIVIDUAL
6,60 months,15.96%,C,C5,Southern Star Photography,8 years,RENT,Not Verified,Dec-11,Fully Paid,...,debt_consolidation,Loan,280xx,NC,Jul-05,85.60%,f,May-16,May-16,INDIVIDUAL
7,36 months,18.64%,E,E1,MKC Accounting,9 years,RENT,Source Verified,Dec-11,Fully Paid,...,car,Car Downpayment,900xx,CA,Jan-07,87.50%,f,Jan-15,Dec-14,INDIVIDUAL
9,60 months,12.69%,B,B5,Starbucks,< 1 year,RENT,Verified,Dec-11,Charged Off,...,other,Building my credit history.,774xx,TX,Sep-04,36.50%,f,Nov-12,Mar-13,INDIVIDUAL
10,60 months,14.65%,C,C3,Southwest Rural metro,5 years,OWN,Not Verified,Dec-11,Fully Paid,...,debt_consolidation,High intrest Consolidation,853xx,AZ,Jan-98,20.60%,f,Jun-13,Dec-15,INDIVIDUAL
11,36 months,12.69%,B,B5,UCLA,10+ years,OWN,Source Verified,Dec-11,Fully Paid,...,debt_consolidation,Consolidation,913xx,CA,Oct-89,67.10%,f,Sep-13,Aug-13,INDIVIDUAL
12,36 months,13.49%,C,C1,Va. Dept of Conservation/Recreation,< 1 year,RENT,Source Verified,Dec-11,Charged Off,...,debt_consolidation,freedom,245xx,VA,Apr-04,91.70%,f,Jul-12,Nov-12,INDIVIDUAL
13,36 months,9.91%,B,B1,Target,3 years,RENT,Source Verified,Dec-11,Fully Paid,...,credit_card,citicard fund,606xx,IL,Jul-03,43.10%,f,Jan-15,May-16,INDIVIDUAL


In [27]:
df['emp_title'].value_counts()

US Army                                     131
Bank of America                             107
IBM                                          65
AT&T                                         57
Kaiser Permanente                            56
                                           ... 
amneal pharmaceuticals                        1
Net Equity Financial                          1
Kernersville Fire Department                  1
The Keith Corp. (empl by county, now Kei      1
Evergreen Center                              1
Name: emp_title, Length: 28027, dtype: int64

In [28]:
"""
If we replace the missing values with the mode for emp_title, the frequency of the mode will increase from 131 to 2517
This could potentially lead to incorrect insights. Therefore, we will be dropping the missing values for emp_title.
"""
df.dropna(subset=['emp_title'], inplace = True)

In [29]:
df_metadata = get_metadata(df)
df_metadata

,column_name,data_type,count_nulls
0,loan_amnt,int64,0
1,funded_amnt,int64,0
2,funded_amnt_inv,float64,0
3,term,object,0
4,int_rate,object,0
5,installment,float64,0
6,grade,object,0
7,sub_grade,object,0
8,emp_title,object,0
9,emp_length,object,55


In [ ]:
df['emp_length'].value_counts()

In [ ]:
# For emp_length we can impute with the mode
df['emp_length'].fillna('10+ years', inplace = True)

In [ ]:
# Converting date columns to datetime format
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%y')
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], format='%b-%y')
df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'], format='%b-%y')

In [ ]:
df['collections_12_mths_ex_med'].value_counts()

In [ ]:
# For the next two columns, since all the values are 0.0 except for Nulls, we can replace the nulls with 0.0
df['collections_12_mths_ex_med'].fillna(0.0, inplace = True)
df['chargeoff_within_12_mths'].fillna(0.0, inplace = True)

In [ ]:
df['title'].value_counts()

In [ ]:
# Since the number of missing values in title is very smmall, we can replace it with the mode
df['title'].fillna('Debt Consolidation', inplace = True)

In [ ]:
df['revol_util'].value_counts()

In [ ]:
df['chargeoff_within_12_mths'].value_counts()

In [ ]:
df_dflt = df[df['loan_status'] == 'Charged Off']
df_dflt.reset_index(inplace=True)
df_dflt.drop('index', axis = 1, inplace = True)
df_dflt.shape

In [ ]:
df_not_dflt = df[df['loan_status'] != 'Charged Off']
df_not_dflt.reset_index(inplace=True)
df_not_dflt.drop('index', axis = 1, inplace = True)
df_not_dflt.shape

In [ ]:
for col in numeric_cols:
    remove_outliers(df_dflt, col)
    remove_outliers(df_not_dflt, col)

In [ ]:
for col in numeric_cols:
    print(f"Descriptive Stats for {col}")
    compare_numerical_column(df_dflt,df_not_dflt, col)
    print(f"Plots for {col}")
    plot_numerical_cols(df_dflt,df_not_dflt, col)

In [ ]:
df_dflt.head()

In [ ]:
df_dflt.describe()

In [ ]:
dflt_dtypes = get_data_types(df_dflt)

In [ ]:
non_dflt_dtypes = get_data_types(df_not_dflt)

In [ ]:
df_dflt.isnull().sum()

In [ ]:
df_dflt['emp_title'].value_counts()

In [ ]:
df_dflt['emp_length'].value_counts()

In [ ]:
df_dflt['pub_rec_bankruptcies'].value_counts()

In [ ]:
df_not_dflt.isnull().sum()

In [ ]:
data_types = get_data_types(df)

In [ ]:
numeric_cols = data_types[data_types['data_type'] != 'object']

In [ ]:
len(numeric_cols)